### ALS on Transactions and CatBoost Stacking
Plus ALS Rating + ALS Bookmarks

In [1]:
import dataset_utils as du
import pandas as pd
import matplotlib.pyplot as plt
import metric_utils as mu
import als_model as als
import json
%matplotlib inline

### Load Data and make Dataset

In [2]:
tr = du.processTransactions(use_cashe=True)
bookmarks = du.processBookmarks()
ratings = du.processRatings()
catalogue = du.processCatalogue()

In [5]:
# random split transactions 70/30 for ALS/CatBoost
from sklearn.model_selection import train_test_split
tr_als, tr_cb = train_test_split(tr, train_size=0.7)

In [8]:
tr_cb

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
2049083,8116,531296,P,4.381100e+07,150,1,90
1634286,1636,115484,S,4.390455e+07,5511,0,50
6032367,2235,206090,S,4.282467e+07,2287,0,99
7484739,3708,76392,S,4.234948e+07,5505,0,50
1549211,388,511615,S,4.392661e+07,6987,5,90
...,...,...,...,...,...,...,...
4198965,4141,91546,S,4.329237e+07,18822,3,99
6770820,9491,464684,R,4.259463e+07,5870,0,50
8413730,9835,244819,S,4.207837e+07,5941,0,50
1327745,7913,497643,S,4.398038e+07,6842,5,90


### Tune Parameters

In [3]:
tr = du.processTransactions(use_cashe=True)
tr_train, tr_val = du.trainvalTransactions(tr, border=0.95)
# these params perform best:
# rating_params = {'bookmarked': 7, 'purchased': 11, 
#                              'watched_movie': 8, 'watched_series': 9, 'watch_failed': 2}

# try this mix
rating_params = {'bookmarked': 5, 'purchased': 10, 
                             'watched_movie': 8, 'watched_series': 9, 'watch_failed': 1}
#ds_train = du.makeTarget(tr_train, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = du.makeTarget(tr_val, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = ds_val[ds_val['tr_rating'] > rating_params['watch_failed']] 
#best_model, best_score, _ = als.tuneALS(ds_train, ds_val, factors = [10, 20, 30])

### Train/Predict

In [ ]:
test_users = du.processTestUsers()
rating_params = {'bookmarked': 5, 'purchased': 11, 
                             'watched_movie': 8, 'watched_series': 9, 'watch_failed': 2}
ds_train_all = du.makeTarget(tr, du.processCatalogue(), du.processBookmarks(), du.processRatings(), 
                             blend_rating=False,
                             params = rating_params)

final_model, _, t_result = als.tuneALS(ds_train_all, ds_val[:1], factors = [30], t_users=test_users)

Test users done.
Elements filtered out from validation
Sparsity: 0.002326
Iteration 1 of 1


In [5]:
len(t_result)

49992

In [6]:
import pickle
def saveSubmit(name, model, results):
    with open('submits/{0}.json'.format(name), 'w') as f:
        json.dump(t_result, f)
    print('Saved results {0}.json'.format(name))

    with open('submits/model_{0}.pkl'.format(name), 'wb') as f:
        pickle.dump(model, f)
    print('Saved model model_{0}.pkl'.format(name))
    
saveSubmit('als_ensure', final_model, t_result)
    


Saved results als_ensure.json
Saved model model_als_ensure.pkl
